In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mul
from scipy import stats
import pickle
from scipy.integrate import quad
from corner import corner
import pandas as pd
from scipy.stats import gaussian_kde
import dynesty as dyn

In [2]:
GRBs = ['GRB210619B', 'GRB210610B', 'GRB210204A', 'GRB201216C', 'GRB200829A', 'GRB200613A', 'GRB190114C', 'GRB180720B', 'GRB180703A', 'GRB171010A', 'GRB160625B', 'GRB160509A', 'GRB150821A', 'GRB150514A', 'GRB150403A', 'GRB150314A', 'GRB141028A', 'GRB140508A', 'GRB140206A', 'GRB131231A', 'GRB131108A', 'GRB130925A', 'GRB130518A', 'GRB130427A', 'GRB120119A', 'GRB100728A', 'GRB091003A', 'GRB090926A', 'GRB090618', 'GRB090328', 'GRB081221', 'GRB080916C']
err_grb = GRBs
grbparam = pd.read_csv(os.getcwd() + '/../data/GRBPARAM.csv', index_col=0)
for grb in err_grb:
    
    
    grbname = grb + '.txt'
    grbname_wtht_ext = grbname.replace('.txt','')


    arr = np.loadtxt(os.getcwd() + '/asciidataof_fig1/32lag/'+grbname)
    data = [arr[:,0], arr[:,1], arr[:,2]]
    x = arr[:,0]
    y = arr[:,1]
    yerr = arr[:,2]

    #Properties of GRB
    E0 = grbparam[grbname.replace('.txt','')].E0
    E0rest = E0
    Erest = arr[:,0]    #in keV
    z_com = grbparam[grbname.replace('.txt','')].redshift #redshift
    H0=67.36 #Hubble constant km -1 Mpc s -1
    omega_m = 0.315
    omega_l = 1 - omega_m
    grb
    #MODELS

    #NULL model
    def nullhp(E, alpha, tau):
        return (1 + z_com)*(tau * ((E ** alpha) - (E0 ** alpha)))
        


    def int_z(z_prime, n):
        integ_fn = lambda z: (1+z)**n / np.sqrt(omega_m * (1+z)**3 + omega_l)
        return quad( integ_fn, a=0, b=z_prime)[0]

    int_z1 = np.asarray(int_z(z_com, 1))
    int_z2 = np.asarray(int_z(z_com, 2))

    #LINEAR model
    def linearhp(E, logEqg, alpha, tau):
        
        e0qg = (E - E0) / (10 ** logEqg)
        
        return - (e0qg * int_z1)/H0 + nullhp(E, alpha, tau)

    #QUADRATIC model
    def quadhp(E, logEqg, alpha, tau):
        e0qg = (E**2 - E0 **2) / ((10 ** logEqg)**2)
        
        return -1.5 * (e0qg * int_z2)/H0 + nullhp(E, alpha, tau)
    #LOG-LIKELIHOODS
    def loglike_null(theta):
        alpha, tau = theta
        
        
        model = nullhp(x,  alpha, tau)
        
        return sum(stats.norm.logpdf(*args) for args in zip(y,model,yerr))
        
        

    def loglike_linear(theta):
        logEqg, alpha, tau = theta
        
        model = linearhp(x, logEqg, alpha, tau)
        
        return sum(stats.norm.logpdf(*args) for args in zip(y,model,yerr))

    def loglike_quad(theta):
        logEqg, alpha, tau = theta
        
        model = quadhp(x, logEqg, alpha, tau)
        
        return sum(stats.norm.logpdf(*args) for args in zip(y,model,yerr))


    alphamin = -1
    alphamax = 1
    taumin = -10
    taumax = 10
    logeqmin = 6
    logeqmax = 19
    #PRIOR DISTRIBUTIONS

    def prior_transform_null(theta):
        alpha, tau = theta
        return [(alphamax - alphamin) * alpha + alphamin, (taumax - taumin) * tau + taumin]

    def prior_transform_linear(theta):
        logEqg, alpha, tau = theta
        return [(logeqmax - logeqmin) * logEqg + logeqmin, (alphamax - alphamin) * alpha + alphamin, (taumax - taumin) * tau + taumin]

    def prior_transform_quadratic(theta):
        logEqg, alpha, tau = theta
        return [(logeqmax - logeqmin) * logEqg + logeqmin, (alphamax - alphamin) * alpha + alphamin, (taumax - taumin) * tau + taumin]

    #Properties of GRB
    E0 = grbparam[grbname.replace('.txt','')].E0
    E0rest = E0
    Erest = arr[:,0]    #in keV
    z_com = grbparam[grbname.replace('.txt','')].redshift #redshift
    threesamplers = ['_null_sampler.pkl', '_linear_sampler.pkl', '_quadratic_sampler.pkl']
    sampler0 = dyn.utils.restore_sampler('./outputs/sampler_saves/' + grb + threesamplers[0])
    sampler1 = dyn.utils.restore_sampler('./outputs/sampler_saves/' + grb + threesamplers[1])
    sampler2 = dyn.utils.restore_sampler('./outputs/sampler_saves/' + grb + threesamplers[2])
    results0 = sampler0.results
    results1 = sampler1.results
    results2 = sampler2.results
    with open('outputs/BF/' + grb + '_results.txt', 'w') as f:
        f.write(str(results0.logz[-1]) + ',' + str(results0.logzerr[-1]) + '\n')
        f.write(str(results1.logz[-1]) + ',' + str(results1.logzerr[-1]) + '\n')
        f.write(str(results2.logz[-1]) + ',' + str(results2.logzerr[-1]) + '\n')

In [3]:
data = []
for grb in GRBs:
    temp = np.loadtxt('./outputs/BF/' + grb + '_results.txt', delimiter=',')[:,0]
    # data.append( [grb, grbparam[grb].E0, grbparam[grb].redshift, *np.exp(temp - temp[0])]  )
    data.append( [grb, grbparam[grb].E0, grbparam[grb].redshift, *(temp - temp[0])]  )
    # data.append( [grb, grbparam[grb].E0, grbparam[grb].redshift, *temp]  )

In [4]:
df = pd.DataFrame(data, columns=['GRB', 'E0 (keV)', 'redshift', 'null', 'linear', 'quadratic']).sort_values('GRB', ascending=False).reset_index(drop=True)

# df.to_latex('./outputs/BE/BE_table.txt', index=False, float_format="%.2f")
# df

In [5]:
df

,GRB,E0 (keV),redshift,null,linear,quadratic
0,GRB210619B,10.0,1.9370,0.0,21.160398,15.285277
1,GRB210610B,30.0,1.1300,0.0,105.721788,91.315365
2,GRB210204A,10.0,0.8760,0.0,31.886442,30.918768
3,GRB201216C,15.0,1.1000,0.0,-1.053512,-0.542373
4,GRB200829A,25.0,1.2500,0.0,187.609165,180.329798
5,GRB200613A,30.0,1.2200,0.0,-0.731348,-0.212957
6,GRB190114C,10.0,0.4250,0.0,148.138624,101.074935
7,GRB180720B,25.0,0.6540,0.0,4.915233,2.184705
8,GRB180703A,20.0,0.6678,0.0,0.862521,7.214614
9,GRB171010A,10.0,0.3285,0.0,18.611524,18.257289


In [6]:
data = []
for i in os.listdir('./outputs/GOF/'):
    # with open('./outputs/GOF/' + i, 'rb') as f:
    #     data.append(np.load(f))
    if i.endswith('_GOF.txt'):
        data.append([i.replace('_GOF.txt', ''), *np.loadtxt('./outputs/GOF/' + i)])
df2 = pd.DataFrame(data, columns=['GRB2', 'Null', 'Linear', 'Quadratic'])
# df2.index = df2['GRB']

df2.sort_values('GRB2', inplace=True, ascending=False)
df2.reset_index(drop=True, inplace=True)
# df2.drop('GRB', axis=1, inplace=True)

In [7]:
df2

,GRB2,Null,Linear,Quadratic
0,GRB210619B,8.571664,6.264282,6.349918
1,GRB210610B,13.316587,1.872561,3.409978
2,GRB210204A,12.062783,8.514449,8.639337
3,GRB201216C,1.338974,1.229487,1.242237
4,GRB200829A,24.293723,6.093917,6.708918
5,GRB200613A,0.635289,0.525552,0.683216
6,GRB190114C,22.080217,5.017552,10.646810
7,GRB180720B,2.434477,1.002046,1.456062
8,GRB180703A,12.826352,12.421402,12.119615
9,GRB171010A,3.034274,0.613378,0.886374


In [8]:
final = pd.concat([df, df2], axis=1).drop('GRB2', axis=1)#.to_latex('./outputs/table.txt', index=False, float_format="%.2f")

In [9]:
final = final[['GRB', 'E0 (keV)', 'redshift', 'null', 'Null', 'linear', 'Linear', 'quadratic', 'Quadratic']]

In [10]:
final

,GRB,E0 (keV),redshift,null,Null,linear,Linear,quadratic,Quadratic
0,GRB210619B,10.0,1.9370,0.0,8.571664,21.160398,6.264282,15.285277,6.349918
1,GRB210610B,30.0,1.1300,0.0,13.316587,105.721788,1.872561,91.315365,3.409978
2,GRB210204A,10.0,0.8760,0.0,12.062783,31.886442,8.514449,30.918768,8.639337
3,GRB201216C,15.0,1.1000,0.0,1.338974,-1.053512,1.229487,-0.542373,1.242237
4,GRB200829A,25.0,1.2500,0.0,24.293723,187.609165,6.093917,180.329798,6.708918
5,GRB200613A,30.0,1.2200,0.0,0.635289,-0.731348,0.525552,-0.212957,0.683216
6,GRB190114C,10.0,0.4250,0.0,22.080217,148.138624,5.017552,101.074935,10.646810
7,GRB180720B,25.0,0.6540,0.0,2.434477,4.915233,1.002046,2.184705,1.456062
8,GRB180703A,20.0,0.6678,0.0,12.826352,0.862521,12.421402,7.214614,12.119615
9,GRB171010A,10.0,0.3285,0.0,3.034274,18.611524,0.613378,18.257289,0.886374


In [11]:
final['null'] = final['null'].round(1)
final['Null'] = final['Null'].round(1)
final['linear'] = final['linear'].round(1)
final['Linear'] = final['Linear'].round(1)
final['quadratic'] = final['quadratic'].round(1)
final['Quadratic'] = final['Quadratic'].round(1)

In [12]:
final

,GRB,E0 (keV),redshift,null,Null,linear,Linear,quadratic,Quadratic
0,GRB210619B,10.0,1.9370,0.0,8.6,21.2,6.3,15.3,6.3
1,GRB210610B,30.0,1.1300,0.0,13.3,105.7,1.9,91.3,3.4
2,GRB210204A,10.0,0.8760,0.0,12.1,31.9,8.5,30.9,8.6
3,GRB201216C,15.0,1.1000,0.0,1.3,-1.1,1.2,-0.5,1.2
4,GRB200829A,25.0,1.2500,0.0,24.3,187.6,6.1,180.3,6.7
5,GRB200613A,30.0,1.2200,0.0,0.6,-0.7,0.5,-0.2,0.7
6,GRB190114C,10.0,0.4250,0.0,22.1,148.1,5.0,101.1,10.6
7,GRB180720B,25.0,0.6540,0.0,2.4,4.9,1.0,2.2,1.5
8,GRB180703A,20.0,0.6678,0.0,12.8,0.9,12.4,7.2,12.1
9,GRB171010A,10.0,0.3285,0.0,3.0,18.6,0.6,18.3,0.9


In [13]:
final.to_latex('./outputs/table.txt', index=False, float_format="%.2f")

In [14]:
temp = final.sort_values('linear', ascending=False)

In [15]:
temp.reset_index(drop=True, inplace=True)

In [19]:
len(temp[temp['linear'] > np.log(100)])

15

In [20]:
len(temp[temp['quadratic'] > np.log(100)])

16